# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,okha,53.5739,142.9478,1.89,86,3,3.15,RU,1727815311
1,1,limon,10.0000,-83.0333,26.94,94,100,2.06,CR,1727815311
2,2,edinburgh of the seven seas,-37.0676,-12.3116,13.68,86,5,9.03,SH,1727815311
3,3,esil,51.9556,66.4042,12.06,58,87,5.19,KZ,1727815274
4,4,fortuna,40.5982,-124.1573,25.52,60,0,4.63,US,1727815311


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = 'Darkgreen',
    scale = 0.75,
    frame_width = 800,
    frame_height = 500    
)

# Display the map
map

ImportError: cannot import name 'param_value_if_widget' from 'panel.depends' (C:\Users\gavpl\anaconda3\envs\dev\lib\site-packages\panel\depends.py)

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filter_temp1 =  city_data_df["Max Temp"] < 27
filter_temp2 =  city_data_df["Max Temp"] > 15
filter_cloud = city_data_df['Cloudiness'] < 60

filtered_df = city_data_df.loc[filter_temp1 & filter_cloud & filter_temp2]

# Drop any rows with null values
final_df = filtered_df.dropna()

# Display sample data
final_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,fortuna,40.5982,-124.1573,25.52,60,0,4.63,US,1727815311
8,8,port mathurin,-19.6833,63.4167,22.41,84,28,9.24,MU,1727815312
21,21,daru,-9.0763,143.2092,25.66,87,53,7.53,PG,1727815313
22,22,pandhana,21.7000,76.2167,25.33,82,11,2.57,IN,1727815313
27,27,kone,-21.0595,164.8658,23.30,68,26,2.54,NC,1727815313


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns_list= ['City_ID','Max Temp','Cloudiness','Wind Speed','Date']
hotel_df = final_df.copy().drop(columns=columns_list)


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
4,fortuna,40.5982,-124.1573,60,US,
8,port mathurin,-19.6833,63.4167,84,MU,
21,daru,-9.0763,143.2092,87,PG,
22,pandhana,21.7000,76.2167,82,IN,
27,kone,-21.0595,164.8658,68,NC,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 5,
    "apiKey": geoapify_key

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fortuna - nearest hotel: No hotel found
port mathurin - nearest hotel: No hotel found
daru - nearest hotel: No hotel found
pandhana - nearest hotel: No hotel found
kone - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
sao joao da barra - nearest hotel: No hotel found
broome - nearest hotel: No hotel found
coquimbo - nearest hotel: No hotel found
prado - nearest hotel: No hotel found
turpan - nearest hotel: No hotel found
minas de marcona - nearest hotel: No hotel found
labrador city - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
idri - nearest hotel: No hotel found
port-vila - nearest hotel: No hotel found
laikit, laikit ii (dimembe) - nearest hotel: No hotel found
tanumah - nearest hotel: No hotel found
guerrero negro - nearest hotel: No hotel found
toliara - nearest hotel: No hotel found
ramos arizpe - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found
marka - nearest hotel: No h

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = 'Darkgreen',
    scale = 0.75,
    frame_width = 800,
    frame_height = 500
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map